In [193]:
import pandas as pd
import json

path = "./data/"
count = 2200

## csv 파일 json 으로 변경

In [194]:
def csv_to_json(name):
    # df = pd.read_csv(path + "data.csv")
    # df.to_json(path + "data.json", orient="records")
    df = pd.read_csv(path + f"tuned_parameters_{name}_{count}.csv")
    df_transposed = df.T
    df_transposed.columns = df_transposed.iloc[0]
    df_transposed.drop(df_transposed.index[0], inplace=True)
    df_transposed.to_csv(f'./tuned_parameters_temp_{name}.csv')
    df_transposed = pd.read_csv(f'./tuned_parameters_temp_{name}.csv')
    df_transposed = df_transposed.drop(["Unnamed: 0", "Accumulated Coverage"], axis=1)

    # 문자열 데이터만 선택
    string_columns = df_transposed.select_dtypes(include=['object'])

    string_columns.columns

    time = ['seed-time', 'uncovered-update-interval', 'max-solver-time', 'batch-time']
    for t in time:
        df_transposed[t] = df_transposed[t].apply(lambda x: str(x).replace('s', ''))
        df_transposed[t] = pd.to_numeric(df_transposed[t], errors='coerce').astype(int)

    # 'sym-files' 열을 공백을 기준으로 분리하고 결과를 새로운 열에 할당
    df_transposed[['sym-files-1', 'sym-files-2']] = df_transposed['sym-files'].str.split(expand=True)
    df_transposed['sym-files-1'] = pd.to_numeric(df_transposed['sym-files-1'], errors='coerce').astype(int)
    df_transposed['sym-files-2'] = pd.to_numeric(df_transposed['sym-files-2'], errors='coerce').astype(int)

    df2 = df_transposed.drop(["sym-files", "Coverage"], axis=1)
    df2["Coverage"] = df_transposed["Coverage"]

    # 문자열 데이터만 선택
    string_columns = df2.select_dtypes(include=['object'])

    df2.to_csv(path + f"tuned_parameters_{name}_{count}_dataset.csv", index=False)

    data_list = []
    for index, row in df2.iterrows():
        config = {}
        for key, value in row.items():
            if key != "Coverage":
                config[key] = value
        data = {
            "id": index,
            "config": config,
            "metric": row["Coverage"]
        }
        data_list.append(data)

    data_list
    json.dump(data_list, open(path + f"tuned_parameters_{name}_{count}.json", 'w'), indent=4)


In [195]:
for name in ["diff", "du", "gawk", "gcal", "ls"]:
    csv_to_json(name)

## effect 구하기

In [196]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
import xgboost as xgb

import shap 

def prepare_data(data_path):
    # 데이터 로드
    data = pd.read_csv(data_path)
    
    # 특성(X)과 타겟(y) 분리
    X = data.drop(['Coverage'], axis=1)
    y = data['Coverage']
    
    # y값에 MinMaxScaler 적용
    y_scaler = MinMaxScaler()
    y_scaled = y_scaler.fit_transform(y.values.reshape(-1, 1)).ravel()
    
    # 레이블 인코딩 적용
    le = LabelEncoder()
    categorical_columns = list(X.select_dtypes(include=['object']).columns)
    for col in categorical_columns:
        X[col] = le.fit_transform(X[col])
    
    # 수치형 변수 스케일링
    scaler = StandardScaler()
    numerical_columns = X.select_dtypes(include=['int64', 'float64']).columns
    for col in numerical_columns:
        X[col] = scaler.fit_transform(X[col].values.reshape(-1, 1))
    
    # 데이터 분할 (7:2:1)
    X_train_val, X_test, y_train_val, y_test = train_test_split(X, y_scaled, test_size=0.1, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)

    return X_train, X_val, X_test, y_train, y_val, y_test, y_scaler, X.columns, X

def evaluate_model(model, X_train, X_val, X_test, y_train, y_val, y_test, y_scaler, model_name):
    # 모델 학습
    model.fit(X_train, y_train)
    
    # 예측
    y_pred_val = model.predict(X_val)
    y_pred_test = model.predict(X_test)
    
    # 검증 세트 평가
    val_mse = mean_squared_error(y_val, y_pred_val)
    val_r2 = r2_score(y_val, y_pred_val)
    
    # 테스트 세트 평가
    test_mse = mean_squared_error(y_test, y_pred_test)
    test_r2 = r2_score(y_test, y_pred_test)
    
    # 원래 스케일로 변환하여 평가
    y_test_original = y_scaler.inverse_transform(y_test.reshape(-1, 1)).ravel()
    y_pred_test_original = y_scaler.inverse_transform(y_pred_test.reshape(-1, 1)).ravel()
    
    test_mse_original = mean_squared_error(y_test_original, y_pred_test_original)
    test_r2_original = r2_score(y_test_original, y_pred_test_original)
    
    return {
        'model_name': model_name,
        'val_mse': val_mse,
        'val_r2': val_r2,
        'test_mse': test_mse,
        'test_r2': test_r2,
        'test_mse_original': test_mse_original,
        'test_r2_original': test_r2_original,
        'model': model
    }

def compare_models(data_path, name):
    # 데이터 준비
    X_train, X_val, X_test, y_train, y_val, y_test, y_scaler, feature_names, X = prepare_data(data_path)
    
    # 모델 정의
    models = {
        'XGBoost': xgb.XGBRegressor(objective='reg:squarederror', n_estimators=1000, learning_rate=0.01, random_state=42),
        # 'RandomForest': RandomForestRegressor(n_estimators=1000, random_state=42),
    }
    
    # 각 모델 평가
    results = []
    feature_importance_dict = {}
    
    for model_name, model in models.items():
        print(f"\nTraining {model_name}...")
        result = evaluate_model(model, X_train, X_val, X_test, y_train, y_val, y_test, y_scaler, model_name)
        results.append(result)
        
        # 특성 중요도 추출 (가능한 모델만)
        if hasattr(model, 'feature_importances_'):
            feature_importance_dict[model_name] = pd.DataFrame({
                'feature': feature_names,
                'importance': model.feature_importances_
            }).sort_values('importance', ascending=False)
    
    # 결과를 DataFrame으로 변환
    results_df = pd.DataFrame(results)
    results_df.set_index('model_name', inplace=True)
    
    # 모델 순위 매기기 (R2 score 기준)
    results_df['rank'] = results_df['test_r2'].rank(ascending=False)
    results_df = results_df.sort_values('rank')

    best_model = results_df.iloc[0]['model']
    explainer = shap.TreeExplainer(best_model)
    shap_values = explainer.shap_values(X)
    scaled_expected_value = explainer.expected_value
    # original_expected_value = y_scaler.inverse_transform([[scaled_expected_value]])[0][0]

    # SHAP 값을 원래 스케일로 변환
    scale_factor = (y_scaler.data_max_ - y_scaler.data_min_)[0]
    original_expected_value = y_scaler.data_min_[0] + (scaled_expected_value * scale_factor)

    print(f"\nExpected Value (Original Scale): {original_expected_value}")   
    scale_factor = (y_scaler.data_max_ - y_scaler.data_min_)[0]
    original_scale_shap_values = shap_values * scale_factor 

    

    df = pd.DataFrame(original_scale_shap_values, columns=X.columns)
    df.to_json(f"./data/{name}_shap_values.json", orient='records')



    # shap.summary_plot(shap_values, X_train)
    
    return results_df, feature_importance_dict, original_expected_value

# 메인 실행 코드
def main(data_path, name):
    print("Starting model comparison...")
    results_df, feature_importance_dict, base = compare_models(data_path, name)
    
    # print("\n=== Model Comparison Results ===")
    # print(results_df)
    
    # 최고 성능 모델 출력
    best_model = results_df.iloc[0]
    print(f"\nBest Model: {results_df.index[0]}")
    print(f"Test R2 Score: {best_model['test_r2']:.4f}")
    print(f"Test MSE: {best_model['test_mse']:.4f}")
    
    # if results_df.index[0] in feature_importance_dict:
    #     print(f"\nTop 10 important features ({results_df.index[0]}):")
    #     print(feature_importance_dict[results_df.index[0]].head(10))


    return results_df, feature_importance_dict, base

all_results = {}
all_results["results"] = []
all_results["names"] = ["diff", "du", "gawk", "gcal", "ls"]
for name in ["diff", "du", "gawk", "gcal", "ls"]:
# for name in ["ls"]:
    print(f"\n{name} dataset:")
    path = f"./data/tuned_parameters_{name}_{count}_dataset.csv"

    results_df, feature_importance_dict, base = main(path, name)

    best_model = results_df.iloc[0]
   
    coverage_set = []

    with open(f"./data/coverage_set_{name}_{count}.txt", 'r') as file:
        data = file.read()
        text_trials = data.split("\n")
        # trials = []
        for trial, index in zip(text_trials, range(len(text_trials))):
            trial = trial.strip("{}").split(", ")
            if len(trial) == 1 and trial[0] == '':
                continue
            else:
                if trial == ['set()']:
                    continue
                else:
                    trial = [int(t) for t in trial]
                    coverage_set += trial
    all_results["results"].append({
        "name": f"{name}_{count}",
        "model": results_df.index[0],
        "r2": best_model["test_r2"],
        "mse": best_model["test_mse"],
        "base": base,
        "max": len(set(coverage_set)),
    })

all_results


diff dataset:
Starting model comparison...

Training XGBoost...

Expected Value (Original Scale): 137.9272609949112

Best Model: XGBoost
Test R2 Score: 0.3810
Test MSE: 0.0140

du dataset:
Starting model comparison...

Training XGBoost...

Expected Value (Original Scale): 346.73585295677185

Best Model: XGBoost
Test R2 Score: 0.7107
Test MSE: 0.0205

gawk dataset:
Starting model comparison...

Training XGBoost...

Expected Value (Original Scale): 1730.7217705249786

Best Model: XGBoost
Test R2 Score: 0.7724
Test MSE: 0.0197

gcal dataset:
Starting model comparison...

Training XGBoost...

Expected Value (Original Scale): 670.5971629023552

Best Model: XGBoost
Test R2 Score: 0.5362
Test MSE: 0.0192

ls dataset:
Starting model comparison...

Training XGBoost...

Expected Value (Original Scale): 373.36785048246384

Best Model: XGBoost
Test R2 Score: 0.6537
Test MSE: 0.0151


{'results': [{'name': 'diff_2200',
   'model': 'XGBoost',
   'r2': 0.38096809750724037,
   'mse': 0.014026365579499638,
   'base': 137.9272609949112,
   'max': 1254},
  {'name': 'du_2200',
   'model': 'XGBoost',
   'r2': 0.7107072710925865,
   'mse': 0.020496644220151773,
   'base': 346.73585295677185,
   'max': 1270},
  {'name': 'gawk_2200',
   'model': 'XGBoost',
   'r2': 0.7723917552347892,
   'mse': 0.01970380450149797,
   'base': 1730.7217705249786,
   'max': 3646},
  {'name': 'gcal_2200',
   'model': 'XGBoost',
   'r2': 0.5362289346771614,
   'mse': 0.01924750992898752,
   'base': 670.5971629023552,
   'max': 4022},
  {'name': 'ls_2200',
   'model': 'XGBoost',
   'r2': 0.6537431654563921,
   'mse': 0.015112777645750442,
   'base': 373.36785048246384,
   'max': 1556}],
 'names': ['diff', 'du', 'gawk', 'gcal', 'ls']}

In [197]:
json.dump(all_results, open("./data/all_results.json", 'w'), indent=4)

## set 데이터셋 정제

In [198]:
import pandas as pd
import pandas as pd
import umap
import json


def main(name):
    print(name)
    # 데이터 읽기
    trials = []
    all_numbers = []
    with open(f'./data/coverage_set_{name}_{count}.txt', 'r') as file:
        data = file.read()
        text_trials = data.split("\n")
        # print("trials", trials[0])
        for trial in text_trials:
            trial = trial.strip("{}").split(", ")
            if len(trial) == 1 and trial[0] == '':
                continue
            else:
                if trial == ['set()']:
                    trials.append([])
                else:
                    trial = [int(t) for t in trial]
                    trials.append(trial)
                    all_numbers += trial

    # 모든 유니크한 숫자 찾기
    
    all_numbers = list(set(all_numbers))
    # all_numbers = sorted(set(num for trial in trials for num in trial))

    # DataFrame 생성
    df = pd.DataFrame(index=range(len(trials)), columns=all_numbers)

    # DataFrame 채우기
    for i, trial in enumerate(trials):
        df.loc[i] = [1 if num in trial else 0 for num in all_numbers]

    # 결과 확인
    # print(df.head())
    # print(f"\nDataFrame shape: {df.shape}")
    X = df.values

    # 자카드 거리 행렬 생성
    # distance_matrix = squareform(pdist(X, metric=jaccard_distance))

    # 하이퍼파라미터 설정
    n_neighbors_list = [5, 15, 50]
    min_dist_list = [0.1, 0.5, 0.9]

    # 결과를 저장할 리스트
    trials = []

    # 결과 시각화를 위한 플롯 설정
    # fig, axes = plt.subplots(len(n_neighbors_list), len(min_dist_list), figsize=(15, 15))
    # fig.suptitle('UMAP projections with different hyperparameters')
    df_list = []
    for i, n_neighbors in enumerate(n_neighbors_list):
        for j, min_dist in enumerate(min_dist_list):
            # UMAP을 사용한 차원 축소
            umap_model = umap.UMAP(n_components=2, metric='jaccard', n_neighbors=n_neighbors, min_dist=min_dist, random_state=42)
            umap_positions = umap_model.fit_transform(X)
            
            # 각 trial의 UMAP 위치 정보 저장
            for trial_idx in range(len(X)):
                if trial_idx >= len(trials):
                    trials.append({"id": trial_idx, "umap_positions": []})
                
                trials[trial_idx]["umap_positions"].append({
                    "n_neighbors": n_neighbors,
                    "min_dist": min_dist,
                    "x": float(umap_positions[trial_idx, 0]),
                    "y": float(umap_positions[trial_idx, 1])
                })
                
            
            # # 결과 시각화
            # ax = axes[i, j]
            # scatter = ax.scatter(umap_positions[:, 0], umap_positions[:, 1],
            #                      c=np.sum(X, axis=1), cmap='viridis', s=5)
            # ax.set_title(f'n_neighbors={n_neighbors}, min_dist={min_dist}')
            # ax.set_xlabel('UMAP1')
            # ax.set_ylabel('UMAP2')

    # plt.tight_layout(rect=[0, 0, 1, 0.96])
    # plt.show()

    with open(f'./data/umap_results_{name}_{count}.json', 'w') as f:
        json.dump({"trials": trials}, f, indent=2)

    # df_list = []


    # for n_neighbors in umap_results:
    #     for min_dist in umap_results[n_neighbors]:
    #         positions = umap_results[n_neighbors][min_dist]
    #         df = pd.DataFrame(positions, columns=['umap_x', 'umap_y'])
    #         df['n_neighbors'] = n_neighbors
    #         df['min_dist'] = min_dist
    #         df_list.append(df)

    # result_df = pd.concat(df_list, ignore_index=True)

    # # DataFrame을 JSON으로 저장
    # result_df.to_json('umap_results.json', orient='records')


for name in ["diff", "du", "gawk", "gcal", "ls"]:
    main(name)

diff


/opt/anaconda3/envs/swt/lib/python3.11/site-packages/umap/umap_.py:1879: UserWarning: gradient function is not yet implemented for jaccard distance metric; inverse_transform will be unavailable
  warn(
/opt/anaconda3/envs/swt/lib/python3.11/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
/opt/anaconda3/envs/swt/lib/python3.11/site-packages/umap/spectral.py:550: UserWarning: Spectral initialisation failed! The eigenvector solver
failed. This is likely due to too small an eigengap. Consider
adding some noise or jitter to your data.

Falling back to random initialisation!
  warn(
/opt/anaconda3/envs/swt/lib/python3.11/site-packages/umap/spectral.py:550: UserWarning: Spectral initialisation failed! The eigenvector solver
failed. This is likely due to too small an eigengap. Consider
adding some noise 

du


/opt/anaconda3/envs/swt/lib/python3.11/site-packages/umap/umap_.py:1879: UserWarning: gradient function is not yet implemented for jaccard distance metric; inverse_transform will be unavailable
  warn(
/opt/anaconda3/envs/swt/lib/python3.11/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
/opt/anaconda3/envs/swt/lib/python3.11/site-packages/umap/umap_.py:1879: UserWarning: gradient function is not yet implemented for jaccard distance metric; inverse_transform will be unavailable
  warn(
/opt/anaconda3/envs/swt/lib/python3.11/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
/opt/anaconda3/envs/swt/lib/python3.

gawk


/opt/anaconda3/envs/swt/lib/python3.11/site-packages/umap/umap_.py:1879: UserWarning: gradient function is not yet implemented for jaccard distance metric; inverse_transform will be unavailable
  warn(
/opt/anaconda3/envs/swt/lib/python3.11/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
/opt/anaconda3/envs/swt/lib/python3.11/site-packages/umap/umap_.py:1879: UserWarning: gradient function is not yet implemented for jaccard distance metric; inverse_transform will be unavailable
  warn(
/opt/anaconda3/envs/swt/lib/python3.11/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
/opt/anaconda3/envs/swt/lib/python3.

gcal


/opt/anaconda3/envs/swt/lib/python3.11/site-packages/umap/umap_.py:1879: UserWarning: gradient function is not yet implemented for jaccard distance metric; inverse_transform will be unavailable
  warn(
/opt/anaconda3/envs/swt/lib/python3.11/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
/opt/anaconda3/envs/swt/lib/python3.11/site-packages/umap/umap_.py:1879: UserWarning: gradient function is not yet implemented for jaccard distance metric; inverse_transform will be unavailable
  warn(
/opt/anaconda3/envs/swt/lib/python3.11/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
/opt/anaconda3/envs/swt/lib/python3.

ls


/opt/anaconda3/envs/swt/lib/python3.11/site-packages/umap/umap_.py:1879: UserWarning: gradient function is not yet implemented for jaccard distance metric; inverse_transform will be unavailable
  warn(
/opt/anaconda3/envs/swt/lib/python3.11/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
/opt/anaconda3/envs/swt/lib/python3.11/site-packages/umap/umap_.py:1879: UserWarning: gradient function is not yet implemented for jaccard distance metric; inverse_transform will be unavailable
  warn(
/opt/anaconda3/envs/swt/lib/python3.11/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
/opt/anaconda3/envs/swt/lib/python3.

In [199]:
for name in ["diff", "du", "gawk", "gcal", "ls"]:
    with open(f"./data/coverage_set_{name}_{count}.txt", 'r') as file:
        data = file.read()
        text_trials = data.split("\n")
        # print("trials", trials[0])

        coverage_set = {}
        coverage_set["trials"] = []
        # trials = []
        for trial, index in zip(text_trials, range(len(text_trials))):
            trial = trial.strip("{}").split(", ")
            if len(trial) == 1 and trial[0] == '':
                # trials.append([])
                coverage_set["trials"].append({
                    "id": index,
                    "branch": []

                })
            else:
                if trial == ['set()']:
                    # trial = []
                    coverage_set["trials"].append({
                        "id": index,
                        "branch": []
                    })
                else:
                    trial = [int(t) for t in trial]
                    coverage_set["trials"].append({
                        "id": index,
                        "branch": trial
                    })
                    # trials.append(trial)
                # trial = [int(t) for t in trial]
                # trials.append(trial)
            
        json.dump(coverage_set, open(f"./data/coverage_set_{name}_{count}.json", 'w'), indent=4)

In [200]:
import json
# 기존 JSON 파일 불러오기
def final(name):
    input_file = f"./data/tuned_parameters_{name}_{count}.json"  # 기존 JSON 파일 이름
    with open(input_file, 'r') as f:
        data = json.load(f)

    # shap value
    shap_values_file = f'./data/{name}_shap_values.json'
    with open(shap_values_file, 'r') as f:
        shap_values = json.load(f)

    # branch coverage   
    branch_coverage_file = f'./data/coverage_set_{name}_{count}.json'
    with open(branch_coverage_file, 'r') as f:
        branch_coverage = json.load(f)

    print(len(branch_coverage["trials"]))


    umap_positions_file = f'./data/umap_results_{name}_{count}.json'
    with open(umap_positions_file, 'r') as f:
        umap_data = json.load(f)

    for i, trial in enumerate(data):
        trial["shap_values"] = shap_values[i]
        trial["branch"] = branch_coverage["trials"][i]["branch"]
        trial["umap_positions"] = umap_data['trials'][i]['umap_positions']
        if trial["metric"] != len(trial["branch"]):
            print(i, trial["metric"], len(trial["branch"]))

        # print()

    # # JSON 파일로 저장
    output_file = f"./data/tuned_parameters_{name}_{count}_final.json"
    with open(output_file, 'w') as f:
        json.dump(data, f, indent=4)

for name in ["diff", "du", "gawk", "gcal", "ls"]:
    final(name)

2201
2201
2201
2201
2201


In [125]:
df = pd.read_csv(f"./data/tuned_parameters_{name}_{count}_dataset.csv")
df["Coverage"] = df["Coverage"].astype(int)
print(df["Coverage"].shape)
df["Coverage"] 

(800,)


0      1513
1      1642
2      1565
3      2195
4      1464
       ... 
795    2584
796    1950
797    1950
798       0
799    1950
Name: Coverage, Length: 800, dtype: int64

In [126]:
with open(f'./data/coverage_set_{name}_{count}.txt', 'r') as file:
    data = file.read()
    text_trials = data.split("\n")
    # print("trials", trials[0])
    trials = []
    for trial in text_trials:
        
        trial = trial.strip("{}").split(", ")
        if len(trial) == 1 and trial[0] == '':
            print("last")
        else:
            if trial == ['set()']:
                trial = 0
                trials.append(trial)
            else:
                trial = [int(t) for t in trial]
                trials.append(len(trial))
print("name", name)
print("length", len(trials))
trials

last
name gawk
length 800


[0,
 0,
 0,
 0,
 1513,
 1642,
 1565,
 2195,
 1464,
 1439,
 1513,
 1446,
 1439,
 1445,
 2042,
 1929,
 1923,
 1542,
 1619,
 1953,
 1531,
 1512,
 2093,
 1547,
 1615,
 1570,
 1612,
 2301,
 1624,
 1591,
 1544,
 2124,
 2032,
 1587,
 1585,
 1590,
 1917,
 2052,
 0,
 2261,
 2396,
 0,
 0,
 2260,
 1692,
 1773,
 2290,
 1774,
 2190,
 2204,
 2257,
 1701,
 1758,
 1751,
 0,
 1791,
 2281,
 2164,
 2362,
 1741,
 1788,
 1767,
 1754,
 0,
 1754,
 2044,
 2494,
 2378,
 2283,
 1829,
 2120,
 2226,
 1831,
 1763,
 1820,
 0,
 2240,
 1720,
 1819,
 2255,
 1722,
 0,
 2322,
 1768,
 2055,
 1723,
 2578,
 2254,
 2333,
 2282,
 1740,
 1735,
 2067,
 0,
 1757,
 1958,
 1737,
 2439,
 2093,
 1964,
 2284,
 2462,
 2134,
 2303,
 2052,
 2256,
 1832,
 2236,
 1800,
 2040,
 2217,
 2297,
 1816,
 2134,
 1821,
 1819,
 1836,
 1848,
 1859,
 2374,
 2561,
 1884,
 1959,
 2196,
 1862,
 1959,
 1907,
 2430,
 2194,
 2456,
 0,
 2206,
 1982,
 2355,
 0,
 1918,
 2325,
 2299,
 1915,
 1864,
 2413,
 1864,
 2579,
 1931,
 1917,
 2300,
 1907,
 1962,
 1907,